In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
from itertools import chain

from model import Encoder, Decoder

In [2]:
data = np.concatenate([np.genfromtxt('../data/dis_forautoencoder_2012_dec_tokyo/day_{}.csv'.format(d), delimiter=',', dtype=np.int32)
                       for d in range(1, 32)], axis=0)

In [3]:
data = data[:, 1:]

In [4]:
pivot = 72
T = 144
data_in = data[:, :pivot]
data_out = data[:, pivot:]

In [5]:
data_size = data.shape[0]

In [6]:
LOC_NUM = 1443
EMBEDDING_DIM = 64
HIDDEN_DIM = 256
LATENT_DIM = 256
N_LAYERS = 2

encoder = Encoder(LOC_NUM, EMBEDDING_DIM, HIDDEN_DIM, LATENT_DIM, N_LAYERS).cuda()
decoder = Decoder(LOC_NUM, EMBEDDING_DIM, HIDDEN_DIM, LATENT_DIM, N_LAYERS).cuda()

In [7]:
optimizer = torch.optim.SGD(chain(encoder.parameters(), decoder.parameters()), lr=1e-2, momentum=0.9)
optim_scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, [2, 4, 6, 8], gamma=0.1)

In [8]:
criteria = nn.CrossEntropyLoss(ignore_index=-1)

In [9]:
batch_size = 64

for epoch in range(1, 11):
    avg_loss = 0.0
    cnt = 0
    np.random.shuffle(data)
    optim_scheduler.step()
    for i in range(0, data_size - batch_size, batch_size):
        optimizer.zero_grad()
        cnt += 1
        x_batch = data_in[i: i + batch_size]
        y_batch = data_out[i: i + batch_size]
        latent_code = encoder(Variable(torch.LongTensor(x_batch)).cuda())
        
        tf_batch = np.ones([batch_size, 1], dtype=np.int32)
        for t in range(T - pivot - 1):
            if np.random.ranf() < 0.5: # teacher forcing
                tf_batch = np.concatenate([tf_batch, y_batch[:, t + 1: t + 2]], axis=1)
            else:
                pred = decoder(latent_code[:, -1], Variable(torch.LongTensor(tf_batch)).cuda(), t + 1)
                prob = nn.functional.softmax(pred[:, t], dim=1).cpu().data.numpy()
                tf_batch = np.concatenate([tf_batch, np.array(list(map(lambda p:np.random.choice(LOC_NUM - 2, 1, p=p), prob))) + 2], axis=1)

        tf_batch = Variable(torch.LongTensor(tf_batch)).cuda()
        pred = decoder(latent_code[:, -1], tf_batch, T - pivot)
        loss = criteria(pred.view(-1, LOC_NUM - 2), torch.clamp(Variable(torch.LongTensor(y_batch - 2)).cuda(), min=-1).view(-1))
        loss.backward()
        avg_loss += float(loss.data[0])
        optimizer.step()
        print('Epoch {:04d}, {:.1f}%, avg_loss={:.4f}'.format(epoch, i * 100 / data_size, avg_loss / cnt), end='\r')
    print('')
    torch.save(encoder, '../results/sadAttenSeq2Seq/encoder_half_tf_sampling')
    torch.save(decoder, '../results/sadAttenSeq2Seq/decoder_half_tf_sampling')

Epoch 0001, 100.0%, avg_loss=2.9095
Epoch 0002, 100.0%, avg_loss=2.0596
Epoch 0003, 100.0%, avg_loss=1.9253
Epoch 0004, 100.0%, avg_loss=1.9132
Epoch 0005, 100.0%, avg_loss=1.9028
Epoch 0006, 100.0%, avg_loss=1.9021
Epoch 0007, 100.0%, avg_loss=1.9000


KeyboardInterrupt: 